# 인공신경망 구성하기

## 실습 목표
----
- pytorch의 주요 기능을 이해한다.
- 주어진 데이터 셋을 활용하여 인공신경망을 설계한다.
- 인공신경망 학습 과정을 코드로 작성하고, 학습이 완료된 모델을 생성한다

## 문제 정의
----

multiclass classifier

## Basic Neural Network


#### [Step1] Load libraries & Datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import  TensorDataset, DataLoader

# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

# 데이터분할
y = df['label']
X = df.drop(['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y)


#### [Step2] Create DataLoader

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

#### [Step3] Set Network Structure

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer    = nn.Linear(4, 16)
        self.hidden_layer1  = nn.Linear(16, 32)
        self.output_layer   = nn.Linear(32, 3)
        self.relu = nn.ReLU()

    def forward(self,x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.output_layer(out)
        return out

#### [Step4] Create Model instance

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)


device = cuda


#### [Step5] Model compile

In [ ]:
# 모델 컴파일
learning_rate = 0.001
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

#### [Step6] Set train loop

In [ ]:
def train_loop(train_loader, model, loss_fn, optimizer):
    size = len(train_loader.dataset)

    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        pred = model(X)

        # 손실 계산
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch * len(X)
        print(f'loss: {loss:>7f}  [{current:>5d}]/{size:5d}')


#### [Step7] Set test loop

In [ ]:
def test_loop(test_loader, model, loss_fn):
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n")

#### [Step8] Run model

In [ ]:
# 모델 실행
epochs = 10

for i in range(epochs) :
    print(f"Epoch {i+1} \n------------------------")
    train_loop(train_dataloader, model, loss, optimizer)
    test_loop(test_dataloader, model, loss)
print("Done!")

Epoch 1 
------------------------
loss: 1.593729  [    0]/  112
loss: 1.431086  [   10]/  112
loss: 1.016690  [   20]/  112
loss: 1.057682  [   30]/  112
loss: 1.159780  [   40]/  112
loss: 1.217506  [   50]/  112
loss: 1.148044  [   60]/  112
loss: 1.120016  [   70]/  112
loss: 1.257463  [   80]/  112
loss: 1.225387  [   90]/  112
loss: 0.966325  [  100]/  112
loss: 1.012281  [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.126636

Epoch 2 
------------------------
loss: 1.165707  [    0]/  112
loss: 1.215374  [   10]/  112
loss: 1.169596  [   20]/  112
loss: 1.099657  [   30]/  112
loss: 1.013388  [   40]/  112
loss: 0.946313  [   50]/  112
loss: 1.126068  [   60]/  112
loss: 1.167127  [   70]/  112
loss: 1.109677  [   80]/  112
loss: 0.982844  [   90]/  112
loss: 1.089505  [  100]/  112
loss: 1.084056  [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.062683

Epoch 3 
------------------------
loss: 1.061782  [    0]/  112
loss: 1.028613  [   10]/  112
loss: 1.064158  